<a href="https://colab.research.google.com/github/sanscurtodavis/tyson_vs_paul/blob/main/paul_vs_tyson_fair_value.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Modules

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

## Create Functions

In [ ]:
# Function to calculate the fighter score based on performance stats
def fighter_score(wins, kos, losses, age, height, reach, experience_factor=1):
    score = (wins * 2) + (kos * 3) - (losses * 1)

    # Age penalty (older age decreases the score)
    age_penalty = max(0, (age - 30) * 0.6)
    score -= age_penalty

    # Reach and height advantage
    reach_advantage = (reach - 71) * 0.2
    score += reach_advantage

    height_advantage = (height - 170) * 0.2
    score += height_advantage

    # Apply the experience factor multiplier
    score *= experience_factor

    return score

# Function to calculate implied probabilities from betting odds
def implied_probability(odds):
    if odds > 0:
        return 100 / (odds + 100)  # for positive odds
    else:
        return -odds / (-odds + 100)  # for negative odds

# Function to calculate the fair value probability
def calculate_fair_value_probability(tyson_score, paul_score, tyson_odds, paul_odds):
    # Normalize the fighter scores
    scaler = MinMaxScaler()
    scaled_scores = scaler.fit_transform(np.array([[tyson_score], [paul_score]]))  # Scaling both scores
    normalized_tyson_score = scaled_scores[0][0]
    normalized_paul_score = scaled_scores[1][0]

    # Calculate implied probabilities from the betting odds
    implied_prob_tyson = implied_probability(tyson_odds)
    implied_prob_paul = implied_probability(paul_odds)

    # Combine the normalized scores with implied probabilities (50% each)
    tyson_combined = (normalized_tyson_score + implied_prob_tyson) / 2
    paul_combined = (normalized_paul_score + implied_prob_paul) / 2

    # Normalize combined probabilities to ensure they sum to 1
    total_combined = tyson_combined + paul_combined
    tyson_prob = tyson_combined / total_combined
    paul_prob = paul_combined / total_combined

    return tyson_prob, paul_prob

## Input Data

In [ ]:
# Tyson's data (Taken from ESPN)
tyson_data = {
    "wins": 50,
    "kos": 44,
    "losses": 6,
    "age": 58,
    "height": 70,
    "reach": 71,
    "experience_factor": 2
}

# Jake Paul's data (Taken from ESPN)
paul_data = {
    "wins": 10,
    "kos": 7,
    "losses": 1,
    "age": 27,
    "height": 73,
    "reach": 76,
    "experience_factor": 0.8
}

## Calculate Fighter Score

In [ ]:
# Calculate fighter scores
tyson_score = fighter_score(tyson_data["wins"], tyson_data["kos"], tyson_data["losses"], tyson_data["age"],
                            tyson_data["height"], tyson_data["reach"], tyson_data["experience_factor"])

paul_score = fighter_score(paul_data["wins"], paul_data["kos"], paul_data["losses"], paul_data["age"],
                           paul_data["height"], paul_data["reach"], paul_data["experience_factor"])

## Calculate Probability

In [ ]:
# Betting odds for Tyson and Paul (Taken from ESPN)
odds_tyson = 170
odds_paul = -210

# Calculate fair value probabilities
tyson_probability, paul_probability = calculate_fair_value_probability(tyson_score, paul_score, odds_tyson, odds_paul)

# Convert to percentage format
tyson_probability, paul_probability = tyson_probability * 100, paul_probability * 100

## Display Results

In [ ]:
# Print the results
print(f"Mike Tyson's fair value probability: {tyson_probability:.2f}%")
print(f"Jake Paul's fair value probability: {paul_probability:.2f}%")
print(f"Sum of probabilities: {tyson_probability + paul_probability:.2f}%")

Mike Tyson's fair value probability: 66.92%
Jake Paul's fair value probability: 33.08%
Sum of probabilities: 100.00%
